In [ ]:
# Import the IDAES Formic Acid Production Model
from flowsheet_NETL_tech import main_steady_state

In [ ]:
# IPC server started for the openLCA database

In [ ]:
# olca-ipc - a Python package for inter-process communication (IPC) with openLCA is installed and imported (v2.4.0) 
# With this package, it is possible to call functions of openLCA and process their results in Python.
import olca_ipc as ipc
import olca_schema as o
import pandas as pd

In [ ]:
#Set up and run IDAES steady state model
m = main_steady_state(optimize=False, dev_mode=True)

In [ ]:
# Establish connection with the IPC server
client = ipc.Client(8080)

In [ ]:
# Access the product system associated with the FA production process for which impact assessment needs to be run
product_system = client.find(model_type=o.ProductSystem, name="FA Production Process")

# Access the impact assessment method required to calculate the LCA metrics
impact_assesment_method = client.find(model_type=o.ImpactMethod, name="TRACI 2.1 (NETL) - AR6 2023")

In [ ]:
# Create a calculation setup for the product system under consideration
calculation_setup = o.CalculationSetup(target=product_system, impact_method=impact_assesment_method)

In [ ]:
#Create a function for transferring simulation results from IDAES to openLCA through input parameter redefinitions

# Note D2O is H2O itself (a duplicate component of H2O). This component is used in the streams and channels that 
# originate from the flue gas entering the process, due to limitations in the current version of the FA production model 
# in IDAES (specifically the electrolyzer unit model)

def data_transfer_idaes_openLCA(m):
    # Transfer FA product flowrate (mol/s)
    redef1 = o.ParameterRedef()
    redef1.name = 'fa_product_out'
    redef1.value = m.fs.acid_separator.acid_outlet.flow_mol[0].value
    
    # Transfer feedwater flowrate (mol/s)
    redef2 = o.ParameterRedef()
    redef2.name = 'feed_water_flow'
    redef2.value = m.fs.h2o_mixer.feed_inlet.flow_mol[0].value
    
    # Transfer CO2 mole fraction in flue gas
    redef3 = o.ParameterRedef()
    redef3.name = 'fg_co2molfrac'
    redef3.value = m.fs.membrane.feed_side_inlet.mole_frac_comp[0,'CO2'].value
    
    # Transfer O2 mole fraction in flue gas
    redef4 = o.ParameterRedef()
    redef4.name = 'fg_o2_molfrac'
    redef4.value = m.fs.membrane.feed_side_inlet.mole_frac_comp[0,'O2'].value

    # Transfer H2O mole fraction in flue gas
    redef5 = o.ParameterRedef()
    redef5.name = 'fg_h2o_molfrac'
    redef5.value = m.fs.membrane.feed_side_inlet.mole_frac_comp[0,'D2O'].value
    
    # Transfer Ar mole fraction in flue gas
    redef6 = o.ParameterRedef()
    redef6.name = 'fg_ar_molfrac'
    redef6.value = m.fs.membrane.feed_side_inlet.mole_frac_comp[0,'Ar'].value
    
    # Transfer N2 mole fraction in flue gas
    redef7 = o.ParameterRedef()
    redef7.name = 'fg_n2_molfrac'
    redef7.value = m.fs.membrane.feed_side_inlet.mole_frac_comp[0,'N2'].value
    
    # Transfer flue gas flowrate (mol/s)
    redef8 = o.ParameterRedef()
    redef8.name = 'fg_flow'
    redef8.value = m.fs.membrane.feed_side_inlet.flow_mol[0].value
    
    # Transfer makeup water flowrate (mol/s)
    redef9 = o.ParameterRedef()
    redef9.name = 'makeup_water_flow'
    redef9.value = m.fs.lean_fa_mixer.makeup_inlet.flow_mol[0].value
    
    # Transfer mole fractions of CO2, N2, O2, H2O, Ar along with the total flowrate (mol/s) of the emission from the CO2 membrane 
    redef10 = o.ParameterRedef()
    redef10.name = 'membrane_emission_co2molfrac'
    redef10.value = m.fs.membrane.feed_side_outlet.mole_frac_comp[0, 'CO2'].value

    redef11 = o.ParameterRedef()
    redef11.name = 'membrane_emission_n2molfrac'
    redef11.value = m.fs.membrane.feed_side_outlet.mole_frac_comp[0, 'N2'].value
    
    redef12 = o.ParameterRedef()
    redef12.name = 'membrane_emission_o2_molfrac'
    redef12.value = m.fs.membrane.feed_side_outlet.mole_frac_comp[0, 'O2'].value
    
    redef13 = o.ParameterRedef()
    redef13.name = 'membrane_emission_h2o_molfrac'
    redef13.value = m.fs.membrane.feed_side_outlet.mole_frac_comp[0, 'D2O'].value
    
    redef14 = o.ParameterRedef()
    redef14.name = 'membrane_emission_ar_molfrac'
    redef14.value = m.fs.membrane.feed_side_outlet.mole_frac_comp[0, 'Ar'].value

    redef15 = o.ParameterRedef()
    redef15.name = 'membrane_emission_totmolflow'
    redef15.value = m.fs.membrane.feed_side_outlet.flow_mol[0].value
    
    # Transfer mole fractions of O2, H2O, along with the total flowrate (mol/s) of the emission from the O2 flash separator
    redef16 = o.ParameterRedef()
    redef16.name = 'o2separator_emission_o2molfrac'
    redef16.value = m.fs.o2_separator.o2_outlet.mole_frac_comp[0, 'O2'].value
    
    redef17 = o.ParameterRedef()
    redef17.name = 'o2separator_emission_totmolflow'
    redef17.value = m.fs.o2_separator.o2_outlet.flow_mol[0].value
    
    redef18 = o.ParameterRedef()
    redef18.name = 'o2separator_emissionh2omolfrac'
    redef18.value = m.fs.o2_separator.o2_outlet.mole_frac_comp[0, 'H2O'].value
    
    # Transfer mole fractions of CO2 ,N2, O2, H2O. Ar along with the total flowrate (mol/s) of the emission from the PSA unit 
    redef19 = o.ParameterRedef()
    redef19.name = 'psa_emission_co2molfrac'
    redef19.value = m.fs.co2_separator.n2_outlet.mole_frac_comp[0, 'CO2'].value

    redef20 = o.ParameterRedef()
    redef20.name = 'psa_emission_n2molfrac'
    redef20.value = m.fs.co2_separator.n2_outlet.mole_frac_comp[0, 'N2'].value
    
    redef21 = o.ParameterRedef()
    redef21.name = 'psa_emission_o2molfrac'
    redef21.value = m.fs.co2_separator.n2_outlet.mole_frac_comp[0, 'O2'].value
    
    redef22 = o.ParameterRedef()
    redef22.name = 'psa_emission_h2omolfrac'
    redef22.value = m.fs.co2_separator.n2_outlet.mole_frac_comp[0, 'D2O'].value
    
    redef23 = o.ParameterRedef()
    redef23.name = 'psa_emission_armolfrac'
    redef23.value = m.fs.co2_separator.n2_outlet.mole_frac_comp[0, 'Ar'].value
    
    redef24 = o.ParameterRedef()
    redef24.name = 'psa_emission_totmolflow'
    redef24.value = m.fs.co2_separator.n2_outlet.flow_mol[0].value

    # Transfer the total electricity required by the FA production process (MWh/day)
    redef25 = o.ParameterRedef()
    redef25.name = 'total_electricity_req'
    redef25.value = m.fs.electricity[0].value
    
    # Specify the parameter redefinitions in the calculation setup
    return [redef1, redef2, redef3, redef4, redef5, redef6, redef7, redef8, redef9, redef10, redef11, redef12, redef13, redef14, redef15, redef16, redef17, redef18, redef19, redef20, redef21, redef22, redef23, redef24, redef25]

# Call the data transfer function to transfer the simulation data from IDAES to openLCA
param_list = data_transfer_idaes_openLCA(m)
calculation_setup.parameter_redefs = param_list

In [ ]:
# Showcase the parameter values
pd.DataFrame({
    'name': [x.name for x in param_list],
    'value': [x.value for x in param_list]
})

In [ ]:
# Run the calculation for the product system
LCA_result = client.calculate(calculation_setup)
LCA_result.wait_until_ready()

In [ ]:
# Initialize empty lists for storing results.
list_of_impact_results_tuples = []
list_of_inventory_results_tuples = []
list_of_impact_categories = []

# Get the life cycle impact assessment and life cycle inventory analysis results
imp_results = LCA_result.get_total_impacts()
inv_results = LCA_result.get_total_flows()
impact_categories = LCA_result.get_impact_categories()

In [ ]:
# Process the inventory results into a dataframe
for inv_result in inv_results:
    inv_tuple = ('baseline', inv_result.to_dict())
    list_of_inventory_results_tuples.append(inv_tuple)

inv_results_df = pd.DataFrame(
    list_of_inventory_results_tuples,
    columns=["scenario", "inv_result_dict"]
)

In [ ]:
# Process the impact results into a dataframe
for imp_result in imp_results:
    imp_tuple = ('baseline', imp_result.to_dict())
    list_of_impact_results_tuples.append(imp_tuple)

imp_results_df = pd.DataFrame(
    list_of_impact_results_tuples,
    columns=["scenario", "imp_result_dict"]
)

exploded_impacts_names = imp_results_df['imp_result_dict'].apply(pd.Series)["impactCategory"].apply(pd.Series)[["name", "refUnit"]]
exploded_values = imp_results_df['imp_result_dict'].apply(pd.Series)["amount"]

# Put it all back together for a final df.
final_results = pd.concat(
    [imp_results_df[["scenario"]],
     exploded_impacts_names,
     exploded_values],
    axis=1
)
final_results.rename(columns={"name": "impact_category"}, inplace=True)
final_results

In [ ]:
# Collect data from openLCA for Plot Global Warming Potential Contribution Results
# Import and use the upstream tree function to access the upstream process names and contributions to the GWP
import olca_ipc.utree as utree
impact_categories = [x for x in LCA_result.get_impact_categories()]

# Get the impact category corresponding to GWP
impact_categories[2].name

In [ ]:
gwp_tree=utree.of(LCA_result, impact_categories[2])

In [ ]:
# Explore the process connections and the individual contributions to the GWP from the upstream tree
print(f"{'FA Production Process Total'} : {gwp_tree.result}") #will give the total GWP.
print(f"{gwp_tree.childs[0].provider.name} : {gwp_tree.childs[0].result}") # will give the contribution of child 0.
print(f"{gwp_tree.childs[0].childs[0].provider.name} : {gwp_tree.childs[0].childs[0].result}") # will give the contribution of child 0 of child 0.
print(f"{gwp_tree.childs[0].childs[1].provider.name} : {gwp_tree.childs[0].childs[1].result}") # will give the contribution of child 1 of child 0.

print(f"{gwp_tree.childs[1].provider.name} : {gwp_tree.childs[1].result}") # will give the contribution of child 1.
print(f"{gwp_tree.childs[2].provider.name} : {gwp_tree.childs[2].result}") # will give the contribution of child 2.

fa_production_process_gwp_contribution = gwp_tree.result - gwp_tree.childs[0].result - gwp_tree.childs[1].result - gwp_tree.childs[2].result
ngcc_electricity_generation_gwp_contribution = gwp_tree.childs[0].result - gwp_tree.childs[0].childs[0].result - gwp_tree.childs[0].childs[1].result

print(f"{gwp_tree.provider.name} : {fa_production_process_gwp_contribution}") # will give the contribution of the FA production process under consideration
print(f"{'NGCC Power Plant Operation'} : {ngcc_electricity_generation_gwp_contribution}") # will give the contribution of the NGCC power plant operation

In [ ]:
# Plot the GWP contribution of different processes
import matplotlib.pyplot as plt

def plot_gwp_contributions(gwp_tree):
    fa_production_process_gwp_contribution = gwp_tree.result - gwp_tree.childs[0].result - gwp_tree.childs[1].result - gwp_tree.childs[2].result
    ngcc_electricity_generation_gwp_contribution = gwp_tree.childs[0].result - gwp_tree.childs[0].childs[0].result - gwp_tree.childs[0].childs[1].result
    list_of_all_contributors = [gwp_tree.provider.name, 'NGCC Power Plant Operation', gwp_tree.childs[0].childs[0].provider.name, gwp_tree.childs[0].childs[1].provider.name, gwp_tree.childs[1].provider.name, 'Water Pretreatment Units']
    gwp_of_all_contributors = [fa_production_process_gwp_contribution, ngcc_electricity_generation_gwp_contribution, gwp_tree.childs[0].childs[0].result, gwp_tree.childs[0].childs[1].result, gwp_tree.childs[1].result , gwp_tree.childs[2].result]

    plt.figure(figsize = (22,6))
    plt.bar(list_of_all_contributors, gwp_of_all_contributors)
    plt.ylabel('GWP Contribution (100 yr): kg CO2 equivalent/kg Formic Acid')
    plt.title('GWP contribution of different processes to the Formic Acid Production Product System')
    plt.tight_layout()
    plt.show()

plot_gwp_contributions(gwp_tree)

In [ ]:
# Dispose the LCA result to release allocated resources of that result
LCA_result.dispose()

In [ ]:
# Perform a sensitivity study by varying the design and operating conditions of the FA production process
from idaes.core.solvers import get_solver
from pyomo.environ import value

# Vary the area of the CO2 capture membrane and simulate the interface iteratively
membrane_area = [1e8, 1.5e8, 2e8, 2.5e8]
LCOP_values = []
GWP_values = []
for mem_area in membrane_area:
    m.fs.membrane.area.fix(mem_area) # cm2
    solver = get_solver("ipopt")
    results = solver.solve(m, tee = False)
    
    calculation_setup.parameter_redefs = data_transfer_idaes_openLCA(m)

    # Run the calculation for the product system
    LCA_result = client.calculate(calculation_setup)
    LCA_result.wait_until_ready()

    gwp_tree=utree.of(LCA_result, impact_categories[2])

    GWP_values.append(gwp_tree.result)
    LCOP_values.append(value(m.fs.costing.cost_of_production)*1e6)

    LCA_result.dispose()

In [ ]:
# Plot the sensitivity study results
# GWP variation with membrane area
plt.figure(figsize = (22,6))
plt.scatter(membrane_area, GWP_values)
plt.xlabel('Membrane Area cm2')
plt.ylabel('GWP Contribution (100 yr): kg CO2 equivalent/kg Formic Acid')
plt.title('GWP values at different carbon capture membrane areas in the Formic Acid Production Product System')
plt.tight_layout()
plt.show()

In [ ]:
# LCOP with membrane area
plt.figure(figsize = (22,6))
plt.scatter(membrane_area, LCOP_values)
plt.xlabel('Membrane Area (cm2)')
plt.ylabel('Levelized cost of product (LCOP) $/kg Formic Acid')
plt.title('LCOP values at different carbon capture membrane areas in the Formic Acid Production Product System')
plt.tight_layout()
plt.show()